In [1]:
# Step 1: Installing required packages
!pip install opencv-python --quiet
!pip install python-barcode --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 kB 3.9 MB/s eta 0:00:00a 0:00:01


In [2]:
# Step 2: Importing libraries
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from IPython.display import Image, display
import barcode
from barcode.writer import ImageWriter
import os
import zipfile

In [3]:
# Step 3: Loading dataset
df = pd.read_csv("/kaggle/input/world-food-facts/en.openfoodfacts.org.products.tsv", 
                 sep='\t', low_memory=False)

In [4]:
# Step 4: Keeping only required columns and shortening dataset
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

columns_needed = [
    'code', 'product_name', 'brands', 'ingredients_text', 'carbohydrates_100g',
    'fat_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'sugars_100g'
]

df = df[[col for col in columns_needed if col in df.columns]].dropna(subset=['code', 'product_name'])
df = df[df.isna().sum(axis=1) <= 2].head(10_000)
# Preview
df.head()

,code,product_name,brands,ingredients_text,carbohydrates_100g,fat_100g,fiber_100g,proteins_100g,salt_100g,sugars_100g
1,0000000004530,Banana Chips Sweetened (Whole),NaN,"Bananas, vegetable oil (coconut oil, corn oil ...",64.29,28.57,3.6,3.57,0.00000,14.29
2,0000000004559,Peanuts,Torn & Glasser,"Peanuts, wheat flour, sugar, rice flour, tapio...",60.71,17.86,7.1,17.86,0.63500,17.86
3,0000000016087,Organic Salted Nut Mix,Grizzlies,"Organic hazelnuts, organic cashews, organic wa...",17.86,57.14,7.1,17.86,1.22428,3.57
4,0000000016094,Organic Polenta,Bob's Red Mill,Organic polenta,77.14,1.43,5.7,8.57,NaN,NaN
5,0000000016100,Breadshop Honey Gone Nuts Granola,Unfi,"Rolled oats, grape concentrate, expeller press...",63.46,18.27,7.7,13.46,NaN,11.54


In [10]:
# Step 5: Generating barcode images for valid product codes
def generate_barcodes_from_codes(df, output_dir="barcodes", zip_filename="barcodes.zip"):
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for idx, row in df.iterrows():
        product_code = str(row['code']).strip()
        
        # Only proceeding with numeric codes
        if not product_code.isdigit():
            #print(f"Skipping invalid code (non-numeric): {product_code}")
            continue
        
        # Ensuring the code is exactly 12 digits (required for EAN-13)
        if len(product_code) != 13:
            continue
        
        try:
            # Creating barcode
            barcode_class = barcode.get_barcode_class('ean13')
            barcode_img = barcode_class(product_code, writer=ImageWriter())
            
            # Saving barcode image
            output_path = os.path.join(output_dir, f"{product_code}")
            barcode_img.save(output_path)
            #print(f"Barcode created for: {product_code}")
        except Exception as e:
            print(f"Error for code {product_code}: {e}")

    # Step 6: Zipping the generated barcode images into a file for download
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for foldername, subfolders, filenames in os.walk(output_dir):
            for filename in filenames:
                file_path = os.path.join(foldername, filename)
                zipf.write(file_path, os.path.relpath(file_path, output_dir))
    
    print(f"✅ All barcodes have been zipped into {zip_filename}.")

#  Run barcode generation and zipping
generate_barcodes_from_codes(df)
            

✅ All barcodes have been zipped into barcodes.zip.
